## In this notebook I outline the process from raw idats to beta values 

<p> I will be following the protocol in Gross et al. Molecular Cell 2016. Breifly, raw beta values will be processed using the minfi R package. Specifically raw betas will be quantile normalized, cell estimates will be calculated and adjusted using the cell count frequency. The resulting matrix will be normalized using BMIQ, the horvath protocol, and the golden standard will be the median of Hannum values for 450K data</p>
<p> After normalization, the human aging models will be applied and the consensus age will be determined.</p>
<p> All the scripts below were done on a high-memory machine (250GB needs to be supplied to load all the idats together to process together. The dataframe manipulations were performed in python on the same node</p>

In [1]:
from __future__ import division
import os
import numpy as np
import random 
import pandas as pd
import itertools
import subprocess
import imp
twto = imp.load_source('twto','/cellar/users/twang/scripts/twang_toolbox.py')
import scipy.stats as stats
import collections

/cellar/users/twang/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
%%R
library(minfi)
#Read in Raw Idats
baseDir <- '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox'
targets <- read.metharray.sheet(baseDir, pattern='2018_02_13_450K_infiniumhdmethSS.csv')
#targets<-targets[which(targets$Basename!="character(0)"),]
rgSet <- read.metharray.exp(targets = targets)
rawMset<-preprocessRaw(rgSet)
rawBetas<-getBeta(rawMset)
detectPval<-detectionP(rgSet)
outf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_RawBetas_192FoxSamples.csv'
write.csv(rawBetas,outf)
outf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_detectionPValues_192FoxSamples.csv'
write.csv(detectPval,outf)


#Read in Hannum (the training dataset for the Hannum model)
baseDir <- '/cellar/users/agross/Data/Hannum_Methylation/idats/'
targets <- read.metharray.sheet(baseDir)
rgSet_H <- read.metharray.exp(targets = targets)
#Read in EPIC for another external control
baseDir2 <- '/cellar/users/agross/Data/Methylation_Controls/EPIC_ITALY'
targets2<-read.delim('/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/EPICItalyTargets.txt',sep='\t',header=TRUE,colClasses="character")
rgSet2 <- read.metharray.exp(targets = targets2)

#Combine the datasets
pData(rgSet)$Sample_Group<-as.character(pData(rgSet)$Sample_Group)
rgSetC <-combine(rgSet,rgSet_H)
rgSet3<-combine(rgSetC,rgSet2)
#Fixes the datatype of the resulting rgSets so we can perform combine
rawMset<-preprocessRaw(rgSet3)
rawBetas<-getBeta(rawMset)
detectPval<-detectionP(rgSet3)
outf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxHannumEpicData_RawBetas.csv'
write.csv(rawBetas,outf)
outf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxHannumEpicData_detectionPValues.csv'
write.csv(detectPval,outf)


#Perform quantile normalization
qnSetC<-preprocessQuantile(rgSetC)

quantBetas<-getBeta(qnSetC)
outf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_QuantNormedBetas.csv'
write.csv(quantBetas,outf)

#estimate cell counts
counts<-estimateCellCounts(rgSetC)
outf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_cellCounts.csv'
write.csv(counts,outf)


qnSet3<-preprocessQuantile(rgSet3)
quantBetas3<-getBeta(qnSet3)
outf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_QuantNormedBetas_AddEpic.csv'
write.csv(quantBetas3,outf)

counts2<-estimateCellCounts(rgSet3)
outf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_cellCounts_AddEpic.csv'
write.csv(counts2,outf)

In [ ]:
#Format The files (back in python)
betas = pd.read_csv('/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_QuantNormedBetas_AddEpic.csv',header=0,index_col=0,sep=',',engine='c')
#Read in cell counts from minfi
cell_counts_f = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_cellCounts_AddEpic.csv'
cell_counts = pd.read_csv(cell_counts_f,header=0,index_col=0)

#Read in the flow sorted data values
hdffannot = '/cellar/users/agross/Data/methylation_annotation.h5'
flowsorteddata = pd.read_hdf(hdffannot,'flow_sorted_data')
cell_type = pd.read_hdf(hdffannot, 'label_map')
#Use the methyatlation decomposition values and the percentages and adjust for the mean using the percentages of estimated cells
n2 = flowsorteddata.groupby(cell_type, axis=1).mean()
avg = n2[cell_counts.columns].dot(cell_counts.T)
cc = avg.mean(1)
adj = (betas - avg).add(cc, axis=0)
adj = adj.dropna(how='all', axis=1)
outf = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_QuantNormedBetas_AddEpic_adjusted.csv'
adj.to_csv(outf,header=True,index=True)
#Use the median of the hannum study to treat as the gold standard for more accurate age estimates
hannum_inds = [x for x in adj.columns if not x.startswith('2008') and not x.startswith('GSM123')]
gs = adj[hannum_inds].median(axis=1)
order = adj.index
gsf='/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/Hannum_Median_450KCpGs_AdjustedQNBetas.csv'
#Just making sure the order of the golden standard is the same as the betas ordering
gs.loc[order].to_csv(gsf,sep=',',header=False,index=True)

The version of R had to be downgraded to make sure WCGNA can install properly and can use the Horvath BMIQ. This is using R-version 3.2.4

In [ ]:
%%R#Back in R
#Normalize Fox quantile normalized betas with Hannum and Epic together to Hannum's median values as the gold standard
library(data.table)
library('WGCNA')
source('/cellar/users/twang/scripts/methylation_scripts/NORMALIZATION.R')
gsf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/Hannum_Median_450KCpGs_AdjustedQNBetas.csv'
gs<-fread(gsf,header=FALSE,sep = ',')
gs<-as.data.frame(gs)
gsv <- gs['V2']
gsvv <- t(gsv) 
betasf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_QuantNormedBetas_AddEpic_adjusted.csv'
betas<-fread(betasf,header=TRUE,sep=',', data.table=TRUE)
betas<-as.data.frame(betas)
betasrm<-betas[, names(betas) !='V1']
betasrm<-t(betasrm)
datan<-BMIQcalibration(betasrm, gsvv)
outf<-'/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxHannumEpic_BMIQ_QuantNormed_AdjustedBetas.csv'
write.csv(datan,file=outf)

In [22]:
#Format The Fox covariates file (back in python)
covariates_file = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/MAMOA CBC methylome biometrics 41618.xlsx'
covariates_fox = pd.read_excel(covariates_file,header=1,index_col=0)
sample_mapping_f = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/2018_02_13_450K_infiniumhdmethSS.csv'
sample_mapping = pd.read_csv(sample_mapping_f,header=0,index_col=None,skiprows=range(7))
covariates_fox = covariates_fox.loc[covariates_fox.index.difference(['Plate2'])]
covariates_fox_samplesonly = covariates_fox[~covariates_fox.index.isnull()].copy(deep=True)
sample_mapping['ID_Chip'] = ['{}_{}'.format(x,y) for x,y in sample_mapping[['Sentrix_ID','Sentrix_Position']].values] 
map_samples = {sample_mapping.loc[ind,'Sample_Name'].replace(' ',''):sample_mapping.loc[ind,'ID_Chip'] for ind in sample_mapping.index}
covariates_fox_samplesonly['SampleName2Map'] = [x.replace('Extra','extra').replace(' ','') for x in covariates_fox_samplesonly.index]
covariates_fox_samplesonly['ID_Chip'] = covariates_fox_samplesonly['SampleName2Map'].map(map_samples)
covariates_fox_samplesonly['Sentrix_ID'] = [x.split('_')[0] for x in covariates_fox_samplesonly['ID_Chip'].values]
covariates_fox_samplesonly['Sentrix_Position'] = [x.split('_')[1] for x in covariates_fox_samplesonly['ID_Chip'].values]
outf = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxCovariates.txt'
covariates_fox_samplesonly.to_csv(outf,sep='\t',header=True,index=True,encoding='utf-8-sig')

In [ ]:
adj_betas = pd.read_csv('/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxHannumEpic_BMIQ_QuantNormed_AdjustedBetas.csv',header=0,index_col=0,sep=',',engine='c')
gsf = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_QuantNormedBetas_AddEpic_adjusted.csv'
gsf = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/Hannum_Median_450KCpGs_AdjustedQNBetas.csv'
gs = pd.read_csv(gsf,sep=',',header=None,index_col=0,engine='c')
adj_betas.columns = gs.index

In [111]:
#Map the Hannum and Epic covariates
outfcovars='/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/children_methylation/data/HannumEpicGSE36054EMTAB4147_Covariates.csv'
covariates_fox_f = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxCovariates.txt'

covars = pd.read_csv(outfcovars,header=0,index_col=0)
covariates_fox = pd.read_csv(covariates_fox_f, sep = '\t',header = 0, index_col = 0)

map_names = {ind:covars.loc[ind,'Sample_Name'] for ind in covars.index}
map_names['F_BL'] = 'FX_BL'
for ind in covariates_fox.index:
    idchp = covariates_fox.loc[ind,'ID_Chip']
    map_names[idchp] = ind

In [45]:
#Rename the columns to reflect the covariates index
adj_betas.index = [map_names[x] for x in adj_betas.index ]

In [ ]:
hannum_epic_f = '/cellar/users/twang/Data/nrnb01_nobackup/dog_methylation/dogs_108/methyldfs_T2/analysis/deduped/HumanComparisons/train_data/AllAdults_BMIQ_DogSpace_5x_imputed_10permissing_30perSampsFilt_AdjBetas_AllSites.h5'
hannum_epic_covs = pd.read_hdf(hannum_epic_f, 'covariates')

In [60]:
#Read in cell counts
cell_counts_f = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxData_cellCounts_AddEpic.csv'
cell_counts = pd.read_csv(cell_counts_f,header=0,index_col=0)
cell_counts.index = [map_names[x] for x in cell_counts.index ]
# Make a index to dataframe mapping
map_study_ind = pd.Series(['Fox']*covariates_fox.shape[0]+['Hannum']*hannum_epic_covs[hannum_epic_covs['study']=='Hannum'].shape[0]+['Epic']*hannum_epic_covs[hannum_epic_covs['study']!='Hannum'].shape[0],
                         index = covariates_fox.index.values.tolist()+hannum_epic_covs[hannum_epic_covs['study']=='Hannum'].index.values.tolist()+hannum_epic_covs[hannum_epic_covs['study']!='Hannum'].index.tolist())

In [ ]:
df_dict = {'adj_betas':adj_betas,'cell_counts':cell_counts,'covariates':covariates_fox,
           'Other_Covariates':hannum_epic_covs,'study_index':map_study_ind,'qn_betas':betas.T,'adj_qn_betas':adj.T}

outhdf = '/cellar/users/twang/Data/nrnb01_nobackup/human_methylation/Fox/files/FoxHannumEpic_BMIQ_QuantNormed_AdjustedBetas.h5'
twto.save_hdf_file(df_dict,outhdf)